In [89]:
import pandas as pd
import ncbi_genome_download as ngd
import os
import pathlib
from Bio import Entrez, SeqIO
import subprocess

In [5]:
aligner = 'bowtie'
threads = '4'

In [6]:
test_kraken = pd.read_csv("testing_samples/M1_kreport.tsv", sep='\t', comment='#')

In [11]:
species_subset = test_kraken.loc[(test_kraken['rank'] == 'species') & (test_kraken['taxReads'] > 500) & (test_kraken['taxReads'] < 510)]

In [12]:
len(species_subset)

1

In [13]:
species = list(species_subset['taxID'])

In [14]:
file_path = 'taxid_list.txt'

with open(file_path, 'w') as file:
    for item in species:
        file.write(str(item) + '\n')

In [15]:
!ncbi-genome-download --formats fasta --section refseq -R representative --taxids taxid_list.txt -p 4 -r 4 -m metadata.tsv all

In [16]:
ngd_metadata = pd.read_csv("metadata.tsv", sep='\t')

In [17]:
ngd_metadata.head()

,assembly_accession,bioproject,biosample,wgs_master,excluded_from_refseq,refseq_category,relation_to_type_material,taxid,species_taxid,organism_name,...,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,local_filename
0,GCF_900143685.1,PRJNA224116,SAMEA4555315,na,na,representative genome,assembly from type material,1871022,1871022,Parolsenella massiliensis,...,Chromosome,Major,Full,2016/12/08,PRJEB18025,NaN,GCA_900143685.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/9...,./refseq/bacteria/GCF_900143685.1/GCF_90014368...


In [18]:
len(ngd_metadata)

1

In [19]:
## Extracting reads, mapping to reference, converting to BAM and creating coverage map
## Short Read

pathlib.Path('./bindex').mkdir(parents=True, exist_ok=True) 

for taxid in ngd_metadata['taxid']:
    koutput = 'testing_samples/M1_koutput.tsv'
    kreport = 'testing_samples/M1_kreport.tsv'
    read1 = 'testing_samples/M1_R1.fq.gz'
    read2 = 'testing_samples/M1_R2.fq.gz'
    output_r1 = 'R1_' + str(taxid) + '.fq'
    output_r2 = 'R2_' + str(taxid) + '.fq'

    refernce_path_gz = list(ngd_metadata.loc[ngd_metadata['taxid'] == taxid]['local_filename'])[0]
    refernce_path = refernce_path_gz.replace('.gz','')
    
    sam_name = 'sam_' + str(taxid) + '.sam'
    
    ## Running Kraken Tools    
    ktools_command = 'extract_kraken_reads.py --fastq-output -t ' + str(taxid)  + ' -k ' + koutput + ' -1 ' + read1 + ' -2 ' + read2 + ' -o ' +  output_r1 + ' -o2 ' + output_r2
    os.system('echo ' + ktools_command)
    os.system(ktools_command)

    ## Running Bowtie2
    gunzip = 'gunzip ' + refernce_path_gz
    bowtie_build = 'bowtie2-build ' + refernce_path + ' ./bindex/' + str(taxid)
    bowtie_map = 'bowtie2 -p ' + threads + ' -x ./bindex/' + str(taxid) + ' -1 ' + output_r1 + ' -2 ' + output_r2 + ' -S ' + sam_name

    os.system('echo ' + gunzip)
    os.system(gunzip)
    os.system('echo ' + bowtie_build)
    os.system(bowtie_build)
    os.system('echo ' + bowtie_map)
    os.system(bowtie_map)

extract_kraken_reads.py --fastq-output -t 1871022 -k testing_samples/M1_koutput.tsv -1 testing_samples/M1_R1.fq.gz -2 testing_samples/M1_R2.fq.gz -o R1_1871022.fq -o2 R2_1871022.fq
PROGRAM START TIME: 03-15-2024 18:53:46
	1 taxonomy IDs to parse
>> STEP 1: PARSING KRAKEN FILE FOR READIDS testing_samples/M1_koutput.tsv
	13.14 million reads processed
	462 read IDs saved
>> STEP 2: READING SEQUENCE FILES AND WRITING READS
	462 read IDs found (6.56 mill reads processed)
	462 read IDs found (6.56 mill reads processed)
	462 reads printed to file
	Generated file: R1_1871022.fq
	Generated file: R2_1871022.fq
PROGRAM END TIME: 03-15-2024 18:55:49
gunzip ./refseq/bacteria/GCF_900143685.1/GCF_900143685.1_PRJEB18025_genomic.fna.gz
bowtie2-build ./refseq/bacteria/GCF_900143685.1/GCF_900143685.1_PRJEB18025_genomic.fna ./bindex/1871022


gzip: ./refseq/bacteria/GCF_900143685.1/GCF_900143685.1_PRJEB18025_genomic.fna already exists;	not overwritten
Building a SMALL index


Settings:
  Output files: "./bindex/1871022.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ./refseq/bacteria/GCF_900143685.1/GCF_900143685.1_PRJEB18025_genomic.fna
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 501783
Using parameters --bmax 376338 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 376338

Renaming ./bindex/1871022.3.bt2.tmp to ./bindex/1871022.3.bt2
Renaming ./bindex/1871022.4.bt2.tmp to ./bindex/1871022.4.bt2
Renaming ./bindex/1871022.1.bt2.tmp to ./bindex/1871022.1.bt2
Renaming ./bindex/1871022.2.bt2.tmp to ./bindex/1871022.2.bt2
Renaming ./bindex/1871022.rev.1.bt2.tmp to ./bindex/1871022.rev.1.bt2
Renaming ./bindex/1871022.rev.2.bt2.tmp to ./bindex/1871022.rev.2.bt2
462 reads; of these:
  462 (100.00%) were paired; of these:
    421 (91.13%) aligned concordantly 0 times
    41 (8.87%) aligned concordantly exactly 1 time
    0 (0.00%) aligned concordantly >1 times
    ----
    421 pairs aligned concordantly 0 times; of these:
      1 (0.24%) aligned discordantly 1 time
    ----
    420 pairs aligned 0 times concordantly or discordantly; of these:
      840 mates make up the pairs; of these:
        808 (96.19%) aligned 0 times
        32 (3.81%) aligned exactly 1 time
        0 (0.00%) aligned >1 times
12.55% overall alignment rate


In [62]:
header = ['percent_frags', 'number_frags_clade', 'number_frags_taxon', 'num_minimizers', 'distinct_minimizers' ,'rank', 'taxid', 'scientific_name']
test_kraken = pd.read_csv("testing_samples/barcode08_newgenfixed_report.tsv", sep='\t', comment='#', names=header)
species_subset = test_kraken.loc[(test_kraken['rank'] == 'S') & (test_kraken['number_frags_taxon'] > 100)]
len(species_subset)
# & (test_kraken['number_frags_taxon'] < 510)

22

In [21]:
!ncbi-genome-download -h

usage: ncbi-genome-download [-h] [-s {refseq,genbank}] [-F FILE_FORMATS]
                            [-l ASSEMBLY_LEVELS] [-g GENERA] [--genus GENERA]
                            [--fuzzy-genus] [-S STRAINS] [-T SPECIES_TAXIDS]
                            [-t TAXIDS] [-A ASSEMBLY_ACCESSIONS]
                            [--fuzzy-accessions] [-R REFSEQ_CATEGORIES]
                            [--refseq-category REFSEQ_CATEGORIES] [-o OUTPUT]
                            [--flat-output] [-H] [-P] [-u URI] [-p N] [-r N]
                            [-m METADATA_TABLE] [-n] [-N] [-v] [-d] [-V]
                            [-M TYPE_MATERIALS]
                            groups

positional arguments:
  groups                The NCBI taxonomic groups to download (default: all).
                        A comma-separated list of taxonomic groups is also
                        possible. For example: "bacteria,viral"Choose from:
                        ['all', 'archaea', 'bacteria', 'fungi',
        

In [71]:
def load_tsv(filename):
    data = []
    with open(filename, 'r') as tsvfile:
        for line in tsvfile:
            data.append(line.strip())
    return data

filename = 'taxid_list.txt'
taxids = load_tsv(filename)
print(taxids)

['5665', '5664', '5679', '5660', '5671', '562', '573', '28197', '296', '2713414', '28901', '546', '584', '400946', '83771', '158836', '470', '549', '548']


In [108]:
def recurrisve_get_genomes(taxid):
    downlaoded_genome = False
    
    try:
        ncbi_down = 'ncbi-genome-download --formats fasta --section refseq -R representative --taxids ' + str(taxid) + ' -n -p 4 -r 4 -m metadata.tsv all'
        output = subprocess.check_output(ncbi_down, shell=True)
        output_str = output.decode('utf-8')
        output_lines = output_str.splitlines()
        if len(output_lines) == 2:
            print('Representative genomes for reseq found')
            print(output_lines[1])
            assembly_acc = output_lines[1].split()[0]
            downlaoded_genome = True
        elif len(output_lines) > 2:
            print('More than one Representative genomes for reseq found')
            print('Number of Genomes found ' + str((len(output_lines)-1)))
            assembly_acc = output_lines[1].split()[0]
            downlaoded_genome = True
    except subprocess.CalledProcessError as e:
        print("Error: No representative genome in refseq")

    if downlaoded_genome == False:

        try:
            ncbi_down = 'ncbi-genome-download --formats fasta --section genbank -R representative --taxids ' + str(taxid) + ' -n -p 4 -r 4 -m metadata.tsv all'
            output = subprocess.check_output(ncbi_down, shell=True)
            output_str = output.decode('utf-8')
            output_lines = output_str.splitlines()
            if len(output_lines) == 2:
                print('Representative genome for genbank found')
                print(output_lines[1])
                assembly_acc = output_lines[1].split()[0]
                downlaoded_genome = True
            elif len(output_lines) > 2:
                print('More than one Representative genomes for genbank found')
                print('Number of Genomes found ' + str((len(output_lines)-1)))
                assembly_acc = output_lines[1].split()[0]
                downlaoded_genome = True
        except subprocess.CalledProcessError as e:
            print("Error: No representative genome in genbank")

    if downlaoded_genome == False:

        try:
            ncbi_down = 'ncbi-genome-download --formats fasta --section refseq --taxids ' + str(taxid) + ' -n -p 4 -r 4 -m metadata.tsv all'
            output = subprocess.check_output(ncbi_down, shell=True)
            output_str = output.decode('utf-8')
            output_lines = output_str.splitlines()
            if len(output_lines) == 2:
                print('Genome for refseq found')
                print(output_lines[1])
                assembly_acc = output_lines[1].split()[0]
                downlaoded_genome = True
            elif len(output_lines) > 2:
                print('More than one genome for refseq found')
                print('Number of Genomes found ' + str((len(output_lines)-1)))
                print(output_lines[1])
                assembly_acc = output_lines[1].split()[0]
                downlaoded_genome = True
        except subprocess.CalledProcessError as e:
            print("Error: No genome in refseq")


    if downlaoded_genome == False:

        try:
            ncbi_down = 'ncbi-genome-download --formats fasta --section genbank --taxids ' + str(taxid) + ' -n -p 4 -r 4 -m metadata.tsv all'
            output = subprocess.check_output(ncbi_down, shell=True)
            output_str = output.decode('utf-8')
            output_lines = output_str.splitlines()
            if len(output_lines) == 2:
                print('Genome for genbank found')
                print(output_lines[1])
                assembly_acc = output_lines[1].split()[0]
                downlaoded_genome = True
            elif len(output_lines) > 2:
                print('More than one Genomes for genbank found')
                print('Number of Genomes found ' + str((len(output_lines)-1)))
                print(output_lines[1])
                assembly_acc = output_lines[1].split()[0]
                downlaoded_genome = True
        except subprocess.CalledProcessError as e:
            print("Error: No genome in genbank")

    if downlaoded_genome == False:
        assembly_acc = 'na'

    return assembly_acc

ERROR: No downloads matched your filter. Please check your options.


Error: No representative genome in refseq


ERROR: No downloads matched your filter. Please check your options.


Error: No representative genome in genbank


ERROR: No downloads matched your filter. Please check your options.


Error: No genome in refseq
Genome for genbank found
GCA_003992435.1	Leishmania mexicana	215-49


'GCA_003992435.1'

In [109]:
taxid_ls = []
assembly_acc_ls = []

for taxid in taxids:
    assembly_acc_ls.append(recurrisve_get_genomes(taxid))
    taxid_ls.append(taxid)

ERROR: No downloads matched your filter. Please check your options.


Error: No representative genome in refseq


ERROR: No downloads matched your filter. Please check your options.


Error: No representative genome in genbank


ERROR: No downloads matched your filter. Please check your options.


...

Error: No representative genome in genbank
More than one genome for refseq found
Number of Genomes found 956
GCF_000770765.1	Proteus mirabilis	Pm-Oxa48
Representative genomes for reseq found
GCF_002252325.1	Wohlfahrtiimonas chitiniclastica	ATCC 51249
Representative genomes for reseq found
GCF_016747875.1	Succinivibrio dextrinosolvens	ASCUSBF53


ERROR: No downloads matched your filter. Please check your options.


Error: No representative genome in refseq


ERROR: No downloads matched your filter. Please check your options.


Error: No representative genome in genbank
More than one genome for refseq found
Number of Genomes found 2604
GCF_000315775.1	Enterobacter hormaechei	08XA1
Representative genomes for reseq found
GCF_008632635.1	Acinetobacter baumannii	K09-14
Representative genomes for reseq found
GCF_019048385.1	Pantoea agglomerans	FDAARGOS 1447
Representative genomes for reseq found
GCF_007632255.1	Klebsiella aerogenes	Ka37751


In [111]:
print(len(assembly_acc_ls))
print(len(taxid_ls))
print(assembly_acc_ls)

19
19
['GCA_003992435.1', 'GCA_020783415.1', 'GCF_000755165.1', 'GCA_003304975.1', 'GCA_003020905.1', 'GCF_000308975.1', 'GCF_000710075.1', 'GCF_900187115.1', 'GCF_015476275.1', 'GCF_011058315.1', 'GCF_001078155.1', 'GCF_904859905.1', 'GCF_000770765.1', 'GCF_002252325.1', 'GCF_016747875.1', 'GCF_000315775.1', 'GCF_008632635.1', 'GCF_019048385.1', 'GCF_007632255.1']


In [119]:
ncbi_down_gen = 'ncbi-genome-download --section genbank --formats fasta -A ' + str(','.join(assembly_acc_ls)) + ' -p 4 -r 4 -m metadata_genbank.tsv all'
os.system(ncbi_down_gen)
ncbi_down_ref = 'ncbi-genome-download --section refseq --formats fasta -A ' + str(','.join(assembly_acc_ls)) + ' -p 4 -r 4 -m metadata_refseq.tsv all'
os.system(ncbi_down_ref)
ngd_metadata_gen = pd.read_csv("metadata_genbank.tsv", sep='\t')
ngd_metadata_ref = pd.read_csv("metadata_refseq.tsv", sep='\t')
print(len(ngd_metadata_gen))
print(len(ngd_metadata_ref))
ngd_metadata_all = pd.concat([ngd_metadata_gen,ngd_metadata_ref])
print(len(ngd_metadata_all))

4
15
19


In [121]:
ngd_metadata_all.head(10)

,assembly_accession,bioproject,biosample,wgs_master,excluded_from_refseq,refseq_category,relation_to_type_material,taxid,species_taxid,organism_name,...,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,local_filename
0,GCA_003304975.1,PRJNA454506,SAMN09009547,QFBG00000000.1,na,na,na,5660,5660,Leishmania braziliensis,...,Chromosome,Major,Full,2018/07/11,IOC-L_3564,NaN,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,./genbank/protozoa/GCA_003304975.1/GCA_0033049...
1,GCA_020783415.1,PRJNA763067,SAMN21422215,na,na,na,na,5664,5664,Leishmania major,...,Chromosome,Major,Full,2021/11/07,ASM2078341v1,NaN,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,./genbank/protozoa/GCA_020783415.1/GCA_0207834...
2,GCA_003992435.1,PRJNA484340,SAMN09984810,RZOC00000000.1,na,na,na,5665,5665,Leishmania mexicana,...,Contig,Major,Full,2019/01/04,ASM399243v1,NaN,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,./genbank/protozoa/GCA_003992435.1/GCA_0039924...
3,GCA_003020905.1,PRJNA437593,SAMN08583618,na,na,na,na,5671,5671,Leishmania infantum,...,Complete Genome,Major,Full,2018/03/27,ASM302090v1,NaN,na,na,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,./genbank/protozoa/GCA_003020905.1/GCA_0030209...
0,GCF_015476275.1,PRJNA224116,SAMN16674530,na,na,representative genome,na,296,296,Pseudomonas fragi,...,Complete Genome,Major,Full,2020/11/15,ASM1547627v1,NaN,GCA_015476275.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,./refseq/bacteria/GCF_015476275.1/GCF_01547627...
1,GCF_008632635.1,PRJNA224116,SAMN12769618,na,na,representative genome,na,470,470,Acinetobacter baumannii,...,Complete Genome,Major,Full,2019/09/23,ASM863263v1,NaN,GCA_008632635.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,./refseq/bacteria/GCF_008632635.1/GCF_00863263...
2,GCF_904859905.1,PRJNA224116,SAMEA3357462,na,na,representative genome,na,546,546,Citrobacter freundii,...,Complete Genome,Major,Full,2020/11/01,MSB1_1H,NaN,GCA_904859905.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/9...,./refseq/bacteria/GCF_904859905.1/GCF_90485990...
3,GCF_007632255.1,PRJNA224116,SAMN12330869,na,na,representative genome,na,548,548,Klebsiella aerogenes,...,Complete Genome,Major,Full,2019/07/30,ASM763225v1,NaN,GCA_007632255.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,./refseq/bacteria/GCF_007632255.1/GCF_00763225...
4,GCF_019048385.1,PRJNA224116,SAMN16357589,na,na,representative genome,assembly from type material,549,549,Pantoea agglomerans,...,Complete Genome,Major,Full,2021/06/28,ASM1904838v1,NaN,GCA_019048385.1,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,./refseq/bacteria/GCF_019048385.1/GCF_01904838...
5,GCF_000308975.1,PRJNA224116,SAMEA2272029,CAJT00000000.1,na,na,na,562,562,Escherichia coli,...,Scaffold,Major,Full,2012/11/09,ASM30897v2,NaN,GCA_000308975.2,identical,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,./refseq/bacteria/GCF_000308975.1/GCF_00030897...


In [135]:
ngd_metadata_all['organism_name']

0              Leishmania braziliensis
1                     Leishmania major
2                  Leishmania mexicana
3                  Leishmania infantum
0                    Pseudomonas fragi
1              Acinetobacter baumannii
2                 Citrobacter freundii
3                 Klebsiella aerogenes
4                  Pantoea agglomerans
5                     Escherichia coli
6                Klebsiella pneumoniae
7                    Proteus mirabilis
8               Aliarcobacter butzleri
9                  Salmonella enterica
10       Succinivibrio dextrinosolvens
11             Enterobacter hormaechei
12    Wohlfahrtiimonas chitiniclastica
13           Chryseobacterium sp. POL2
14               Leishmania panamensis
Name: organism_name, dtype: object

In [180]:
## Extracting reads, mapping to reference, converting to BAM and creating coverage map
## Long Read

species_name_ls = []
taxid_ls = []
read_mapped_count = []



pathlib.Path('./readsubsets').mkdir(parents=True, exist_ok=True)
pathlib.Path('./tmp_files').mkdir(parents=True, exist_ok=True)
pathlib.Path('./alignments').mkdir(parents=True, exist_ok=True)

#minimap2 -ax sr ${ref} ${fastq1} ${fastq1} > ${sample}.sam 

for taxid in ngd_metadata_all['taxid']:
    
    pathlib.Path('./readsubsets/taxid').mkdir(parents=True, exist_ok=True)

    species_name = list(ngd_metadata_all.loc[ngd_metadata_all['taxid'] == taxid]['organism_name'])[0].replace(" ", "_")

    pathlib.Path('./alignments/' + species_name).mkdir(parents=True, exist_ok=True)
    
    koutput = 'testing_samples/rural4/rural_4_out.tsv'
    kreport = 'testing_samples/rural4/rural_4_report.tsv'
    read1 = 'testing_samples/rural4/rural_4_readsubset.fastq'
    output_r1 = './readsubsets/taxid/R1_' + str(taxid) + '.fq'

    refernce_path_gz = list(ngd_metadata_all.loc[ngd_metadata_all['taxid'] == taxid]['local_filename'])[0]
    refernce_path = refernce_path_gz.replace('.gz','')
    
    sam_name = './tmp_files/sam_' + species_name + '.sam'
    bam_name = './tmp_files/bam_' + species_name + '.bam'
    sort_bam_name = './alignments/' + species_name + '/sorted_bam_' + species_name + '.bam'
    
    ## Running Kraken Tools    
    ktools_command = 'extract_kraken_reads.py --include-children --fastq-output -t ' + str(taxid)  + ' -r ' + kreport + ' -k ' + koutput + ' -1 ' + read1 + ' -o ' +  output_r1
    os.system('echo ' + ktools_command)
    os.system(ktools_command)

    ## Running Minimap
    gunzip = 'gunzip ' + refernce_path_gz
    minimap2_map = 'minimap2 -ax lr:hq ' + refernce_path + ' ' + output_r1 + ' > ' + sam_name
    samtools_view = 'samtools view -c -F 260 ' + sam_name
    samtools_sam2bam = 'samtools view -S -b ' + sam_name + ' -o ' + bam_name
    samtools_bamsort = 'samtools sort ' + bam_name + ' -o ' + sort_bam_name
    samtools_indexbam = 'samtools index ' + sort_bam_name
    
    os.system('echo ' + gunzip)
    os.system(gunzip)
    os.system('echo ' + minimap2_map)
    os.system(minimap2_map)
    os.system('echo ' + samtools_view)
    os.system(samtools_view)
    os.system('echo ' + samtools_sam2bam)
    os.system(samtools_sam2bam)
    os.system('echo ' + samtools_bamsort)
    os.system(samtools_bamsort)
    os.system('echo ' + samtools_indexbam)
    os.system(samtools_indexbam)

    output = subprocess.check_output(samtools_view, shell=True)
    output_str = output.decode('utf-8')
    species_name_ls.append(species_name)
    taxid_ls.append(taxid)
    read_mapped_count.append(output_str.split()[0])

extract_kraken_reads.py --include-children --fastq-output -t 5660 -r testing_samples/rural4/rural_4_report.tsv -k testing_samples/rural4/rural_4_out.tsv -1 testing_samples/rural4/rural_4_readsubset.fastq -o ./readsubsets/taxid/R1_5660.fq
PROGRAM START TIME: 04-17-2024 18:20:44
>> STEP 0: PARSING REPORT FILE testing_samples/rural4/rural_4_report.tsv
	1 taxonomy IDs to parse
>> STEP 1: PARSING KRAKEN FILE FOR READIDS testing_samples/rural4/rural_4_out.tsv
	0.02 million reads processed
	0 read IDs saved
>> STEP 2: READING SEQUENCE FILES AND WRITING READS
	0 read IDs found (0.00 mill reads processed)
	0 reads printed to file
	Generated file: ./readsubsets/taxid/R1_5660.fq
PROGRAM END TIME: 04-17-2024 18:20:44
gunzip ./genbank/protozoa/GCA_003304975.1/GCA_003304975.1_IOC-L_3564_genomic.fna.gz


gzip: ./genbank/protozoa/GCA_003304975.1/GCA_003304975.1_IOC-L_3564_genomic.fna.gz: No such file or directory
[M::mm_idx_gen::0.428*0.99] collected minimizers
[M::mm_idx_gen::0.486*1.22] sorted minimizers
[M::main::0.486*1.22] loaded/built the index for 1029 target sequence(s)
[M::mm_mapopt_update::0.514*1.20] mid_occ = 50
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 1029
[M::mm_idx_stat::0.532*1.20] distinct minimizers: 2821423 (79.18% are singletons); average occurrences: 1.237; average spacing: 10.891; total length: 38003648
[M::main] Version: 2.27-r1193
[M::main] CMD: minimap2 -ax lr:hq ./genbank/protozoa/GCA_003304975.1/GCA_003304975.1_IOC-L_3564_genomic.fna ./readsubsets/taxid/R1_5660.fq
[M::main] Real time: 0.537 sec; CPU: 0.642 sec; Peak RSS: 0.167 GB


samtools view -c -F 260 ./tmp_files/sam_Leishmania_braziliensis.sam
0
samtools view -S -b ./tmp_files/sam_Leishmania_braziliensis.sam -o ./tmp_files/bam_Leishmania_braziliensis.bam
samtools sort ./tmp_files/bam_Leishmania_braziliensis.bam -o ./alignments/Leishmania_braziliensis/sorted_bam_Leishmania_braziliensis.bam
samtools index ./alignments/Leishmania_braziliensis/sorted_bam_Leishmania_braziliensis.bam
extract_kraken_reads.py --include-children --fastq-output -t 5664 -r testing_samples/rural4/rural_4_report.tsv -k testing_samples/rural4/rural_4_out.tsv -1 testing_samples/rural4/rural_4_readsubset.fastq -o ./readsubsets/taxid/R1_5664.fq
PROGRAM START TIME: 04-17-2024 18:20:45
>> STEP 0: PARSING REPORT FILE testing_samples/rural4/rural_4_report.tsv
	2 taxonomy IDs to parse
>> STEP 1: PARSING KRAKEN FILE FOR READIDS testing_samples/rural4/rural_4_out.tsv
	0.02 million reads processed
	7 read IDs saved
>> STEP 2: READING SEQUENCE FILES AND WRITING READS
	7 read IDs found (0.02 mill read

gzip: ./genbank/protozoa/GCA_020783415.1/GCA_020783415.1_ASM2078341v1_genomic.fna.gz: No such file or directory
[M::mm_idx_gen::0.374*0.98] collected minimizers
[M::mm_idx_gen::0.432*1.24] sorted minimizers
[M::main::0.432*1.24] loaded/built the index for 36 target sequence(s)
[M::mm_mapopt_update::0.456*1.23] mid_occ = 50
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 36
[M::mm_idx_stat::0.472*1.22] distinct minimizers: 2982961 (96.66% are singletons); average occurrences: 1.095; average spacing: 10.057; total length: 32854833
[M::worker_pipeline::0.480*1.22] mapped 7 sequences
[M::main] Version: 2.27-r1193
[M::main] CMD: minimap2 -ax lr:hq ./genbank/protozoa/GCA_020783415.1/GCA_020783415.1_ASM2078341v1_genomic.fna ./readsubsets/taxid/R1_5664.fq
[M::main] Real time: 0.487 sec; CPU: 0.593 sec; Peak RSS: 0.165 GB


samtools view -c -F 260 ./tmp_files/sam_Leishmania_major.sam
3
samtools view -S -b ./tmp_files/sam_Leishmania_major.sam -o ./tmp_files/bam_Leishmania_major.bam
samtools sort ./tmp_files/bam_Leishmania_major.bam -o ./alignments/Leishmania_major/sorted_bam_Leishmania_major.bam
samtools index ./alignments/Leishmania_major/sorted_bam_Leishmania_major.bam
extract_kraken_reads.py --include-children --fastq-output -t 5665 -r testing_samples/rural4/rural_4_report.tsv -k testing_samples/rural4/rural_4_out.tsv -1 testing_samples/rural4/rural_4_readsubset.fastq -o ./readsubsets/taxid/R1_5665.fq
PROGRAM START TIME: 04-17-2024 18:20:46
>> STEP 0: PARSING REPORT FILE testing_samples/rural4/rural_4_report.tsv
	2 taxonomy IDs to parse
>> STEP 1: PARSING KRAKEN FILE FOR READIDS testing_samples/rural4/rural_4_out.tsv
	0.02 million reads processed
	16 read IDs saved
>> STEP 2: READING SEQUENCE FILES AND WRITING READS
	16 read IDs found (0.02 mill reads processed)
	16 reads printed to file
	Generated file

...

gzip: ./refseq/bacteria/GCF_011058315.1/GCF_011058315.1_ASM1105831v1_genomic.fna.gz: No such file or directory
[M::mm_idx_gen::0.041*0.96] collected minimizers
[M::mm_idx_gen::0.052*1.23] sorted minimizers
[M::main::0.052*1.23] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.055*1.22] mid_occ = 50
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.057*1.21] distinct minimizers: 311886 (98.15% are singletons); average occurrences: 1.040; average spacing: 9.996; total length: 3243462
[M::worker_pipeline::0.065*1.12] mapped 3 sequences
[M::main] Version: 2.27-r1193
[M::main] CMD: minimap2 -ax lr:hq ./refseq/bacteria/GCF_011058315.1/GCF_011058315.1_ASM1105831v1_genomic.fna ./readsubsets/taxid/R1_2713414.fq
[M::main] Real time: 0.068 sec; CPU: 0.076 sec; Peak RSS: 0.041 GB


PROGRAM START TIME: 04-17-2024 18:21:00
>> STEP 0: PARSING REPORT FILE testing_samples/rural4/rural_4_report.tsv
	1 taxonomy IDs to parse
>> STEP 1: PARSING KRAKEN FILE FOR READIDS testing_samples/rural4/rural_4_out.tsv
	0.02 million reads processed
	2 read IDs saved
>> STEP 2: READING SEQUENCE FILES AND WRITING READS
	2 read IDs found (0.01 mill reads processed)
	2 reads printed to file
	Generated file: ./readsubsets/taxid/R1_5679.fq
PROGRAM END TIME: 04-17-2024 18:21:00
gunzip ./refseq/protozoa/GCF_000755165.1/GCF_000755165.1_ASM75516v1_genomic.fna.gz


gzip: ./refseq/protozoa/GCF_000755165.1/GCF_000755165.1_ASM75516v1_genomic.fna.gz: No such file or directory


samtools view -c -F 260 ./tmp_files/sam_Leishmania_panamensis.sam
2
samtools view -S -b ./tmp_files/sam_Leishmania_panamensis.sam -o ./tmp_files/bam_Leishmania_panamensis.bam
samtools sort ./tmp_files/bam_Leishmania_panamensis.bam -o ./alignments/Leishmania_panamensis/sorted_bam_Leishmania_panamensis.bam
samtools index ./alignments/Leishmania_panamensis/sorted_bam_Leishmania_panamensis.bam


[M::mm_idx_gen::0.340*0.98] collected minimizers
[M::mm_idx_gen::0.391*1.23] sorted minimizers
[M::main::0.391*1.23] loaded/built the index for 35 target sequence(s)
[M::mm_mapopt_update::0.410*1.22] mid_occ = 50
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 35
[M::mm_idx_stat::0.426*1.21] distinct minimizers: 2922105 (98.44% are singletons); average occurrences: 1.027; average spacing: 10.226; total length: 30688794
[M::worker_pipeline::0.428*1.21] mapped 2 sequences
[M::main] Version: 2.27-r1193
[M::main] CMD: minimap2 -ax lr:hq ./refseq/protozoa/GCF_000755165.1/GCF_000755165.1_ASM75516v1_genomic.fna ./readsubsets/taxid/R1_5679.fq
[M::main] Real time: 0.434 sec; CPU: 0.523 sec; Peak RSS: 0.149 GB


In [181]:
read_counts_df = pd.DataFrame()
read_counts_df['Species'] = species_name_ls
read_counts_df['TaxID'] = taxid_ls
read_counts_df['Mapped_Reads'] = read_mapped_count
read_counts_df.to_csv('Rural4_pathogen_confirmation.csv')